In [1]:
%%HTML
<link rel="stylesheet" href="https://doc.splicemachine.com/jupyter/css/custom.css">

In [ ]:
import os
os.environ['JDBC_HOST'] = 'jrtest01-splice-hregion'

<link rel="stylesheet" href="https://doc.splicemachine.com/zeppelin/css/zepstyles.css" />

# Introduction
Splice Machine allows for the creation of custom functions and procedures that you can deploy to your database.  Once deployed, these functions and procedures can be called as needed from your SQL.  This notebook will show you how this is done.

(Important note: when creating custom functions and procedures as described here, the code must be in Java.)



## Your Stored Procedure Development Environment

The _environment_ in which your stored procedure code is developed may change depending upon your need.  For example, while fleshing out the actual functionality of your custom code, we recommend that you do this on a local (standalone) Splice Machine environment.  You can either build this from our open source code, or download our ready-to-use standalone version.  You can find details on these options at https://www.splicemachine.com/get-started/ 

Once you have refined your code in a local environment and it behaves to your satisfaction, you can then re-deploy the developed code to your target instance. 

This notebook covers all of these steps, though as you will see, these steps occur OUTSIDE of an actual Notebook when running in a local environment.  At times you will see `<your-file-name>` in the examples shown.  These should be replaced with your specific file name, etc. accordingly, including the removal of the brackets.


## Local Development of the Custom Code in an IDE

We highly recommend that the initial development of the custom code is done in your local environment (e.g. Mac or Linux).  This way you can develop and test the code in a quick cycle, testing and tuning as needed, before deploying to a full cluster.  The steps described here will be for a Mac, but the process is generally the same for Linux.

Using an Integrated Development Environment (IDE) is usually the best approach when developing code, since development is straightforward, as is the generation of the desired output - a JDK 1.8-compatible JAR file.  We'll be using IntelliJ IDEA here but again most IDE's should work as well.

### Setting Up Your Project

Inside of IntelliJ IDEA, do the following:
1. Select *File -> New -> Project...*
2. Choose *Java* with 1.8 as the Project SDK, then select *Next*.
3. Skip creating from a template unless you prefer to use one, then select *Next*.
4. Give your project a name and location (such as *CompanyACustom1*).
5. If desired, create a new package (*File -> New -> Package*), such as `com.splicemachine.companya`.

### Configuring to output a JAR file

Inside of IntelliJ IDEA, do the following:
1. Select *File -> Project Structure...*.
2. Select *Artifacts*, then click *+*.
3. Choose *JAR -> From modules with dependencies*.
3. Select *Build on make*, then *OK*.

### Coding your function or procedure

1. Create a file: right-click on your src or package folder and select *New -> Java Class*.
2. Paste in code similar to the following:

```
package com.splicemachine.companya;

import java.sql.SQLException;
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.PreparedStatement;


public class Custom1 {


    public static void customIncrement () throws SQLException {
        try (Connection con = DriverManager.getConnection("jdbc:default:connection")) {

            String incrementSQL = "update SPLICE.INCREMENTTEST set i = i + 1";

            try (PreparedStatement ps = con.prepareStatement(incrementSQL)) {
                int resultSet = ps.executeUpdate();
            } catch (SQLException e) {
                con.rollback();
                e.printStackTrace();
                throw e;
            }
        }
    }
}
```

### Build your JAR file
1. Select _Build -> Artifacts... JAR_.
2. Use your command line interface to navigate to where the JAR file is being created, e.g. `out/artifacts/CompanyACustom1_jar/Custom1.jar`.

You are now ready to deploy your JAR file to test!


## Development Guidelines
Before we develop the code and deploy the JAR file, it is worth going through some simple best practices:

1. For code organization it is useful to use a package for the code, e.g. `com.splicemachine.yourcompanyname`.
2. Notice you don't need a *main* code block in your class - you will be calling the class's specific method directly.
3. This example is a stored procedure, therefore it is not returning any results; A function would require a return result.
4. You will need to ensure your method throws an `SQLException` if SQL is being executed.  Be sure to use *try/catch* blocks similar to the example shown here to deal with possible run-time exceptions.


## Local Deployment of the JAR file

It's now time to deploy into your development environment.  Make sure it's up and running first.  Then carefully edit and execute the commands below IN YOUR LOCAL ENVIRONMENT (not in Jupyter) to align to your custom procedure and environment:

```
CALL SQLJ.INSTALL_JAR('<your-full-path-to>/Custom1.jar', 'SPLICE.Custom1', 0);
CALL SYSCS_UTIL.SYSCS_SET_GLOBAL_DATABASE_PROPERTY('derby.database.classpath', 'SPLICE.custom1');
CREATE PROCEDURE SPLICE.customincrement() PARAMETER STYLE JAVA MODIFIES SQL DATA LANGUAGE JAVA DYNAMIC RESULT SETS 0 EXTERNAL NAME 'com.splicemachine.companya.Custom1.customIncrement';
```


## Locally Set up and test the custom procedure

First we set up the table our stored procedure will operate on.  Again for this part we will do this IN YOUR LOCAL ENVIRONMENT:
```
create table incrementtest (i int);
insert into incrementtest values 1;
```

Now we can see the stored procedure in action:

```
call customincrement();
select * from incrementtest;
```


## Deployment of the JAR file to your Service cluster
Once you have confirmed that the code works to your satisfaction, you can now take that same JAR file and deploy it to your Splice Machine cluster on the cloud.  Here are the steps we will follow:

1. Copy the JAR file you already built to S3; your Service cluster will be able to access it from here.
2. Deploy the JAR file similarly as before.
3. Test and run the custom procedure.

These last two steps can now be run using the `%%sql` magic in Jupyter, so you should edit the recommended steps and run them as shown below.

### Copy the JAR file to S3

First make sure the `aws` command tool installed in the local environment where you have your JAR file.  If not, see http://docs.aws.amazon.com/cli/latest/userguide/installing.html

Once `aws` is installed, you can run the following commands in your local environment:

```
aws s3 cp <your-file-name> s3://<your-bucket-name>/<your-JAR-file-name>
aws s3api put-object-acl --bucket <your-bucket-name> --grant-read emailAddress="dbaas@splicemachine.com"  --key=<your-JAR-file-name> --region us-east-1
```

### Set up the custom procedure on your Service cluster

In [ ]:
%%sql 
-- installing the jar and defining the procedure
-- edit these steps for your environment
CALL SQLJ.INSTALL_JAR('S3://<your-bucket-name>/<your-jar-file-name>', 'SPLICE.Custom1', 0);
CALL SYSCS_UTIL.SYSCS_SET_GLOBAL_DATABASE_PROPERTY('derby.database.classpath', 'SPLICE.custom1');
CREATE PROCEDURE SPLICE.customincrement() PARAMETER STYLE JAVA MODIFIES SQL DATA LANGUAGE JAVA DYNAMIC RESULT SETS 0 EXTERNAL NAME 'com.splicemachine.companya.Custom1.customIncrement';

### Run the stored procedure on your Service cluster

In [ ]:
%%sql 
create table incrementtest (i int);
insert into incremenenttest values 1;
call customincrement();
select * from incrementtest;

## Where to go next

Now, let's look at how to [Deploy Apps with Spring](./i.%20Deploying%20Applications%20with%20Spring.ipynb)